In [1]:
# Imports PySpark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Initialisation de la session Spark
spark = SparkSession.builder \
    .appName("BTC_Silver") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/19 20:45:29 WARN Utils: Your hostname, DESKTOP-3IKPR1U, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
26/01/19 20:45:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/19 20:45:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
# Lecture du fichier Parquet Bronze
bronze_path = "/mnt/c/Users/hp/desktop/BTC-End2End-Platform/ml/data/bronze/btc_minute_data.parquet"
df_broze = spark.read.parquet(bronze_path)

# Afficher un aperçu
df_broze.show(5)
df_broze.printSchema()


+-------------------+--------+--------+--------+--------+-------+--------------------+------------------+----------------+---------------------+----------------------+------+
|          open_time|    open|    high|     low|   close| volume|          close_time|quote_asset_volume|number_of_trades|taker_buy_base_volume|taker_buy_quote_volume|ignore|
+-------------------+--------+--------+--------+--------+-------+--------------------+------------------+----------------+---------------------+----------------------+------+
|2026-01-19 09:53:00|93046.35|93046.35|92997.65| 93001.8|9.10532|2026-01-19 09:53:...|    847027.2616431|            2024|              2.08878|        194272.7849138|     0|
|2026-01-19 09:54:00| 93001.8|93022.23|93001.79|93014.94|5.62556|2026-01-19 09:54:...|    523245.8306426|            1064|              3.42885|        318907.7899883|     0|
|2026-01-19 09:55:00|93014.95|93042.27|93014.95|93022.36|3.74306|2026-01-19 09:55:...|    348227.1952126|            1863|   